In [2]:
import numpy as np
%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib.pyplot as plt
import matplotlib as mpl
# mpl.rcParams['text.usetex'] = True
mpl.rcParams['mathtext.fontset'] = 'stix'
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.sans-serif'] = 'stix'
import sympy
sympy.init_printing()
from scipy import integrate

### Documentación

**`%matplotlib inline`**: Permite que los gráficos generados con `matplotlib` se muestren directamente en el notebook, sin necesidad de abrir una ventana externa.

**`%config InlineBackend.figure_format='retina'`**: Mejora la resolución de las figuras al establecer el formato de las imágenes en "retina" (2x resolución). 

**Configuraciones en `mpl.rcParams`**:
   - **`mpl.rcParams['mathtext.fontset'] = 'stix'`**: Configura `matplotlib` para usar el conjunto de fuentes STIX para símbolos matemáticos, mejorando la apariencia y consistencia de las ecuaciones en los gráficos.
   - **`mpl.rcParams['font.family'] = 'serif'` y `mpl.rcParams['font.sans-serif'] = 'stix'`**: Establecen el tipo de fuente global para las gráficas. La fuente "serif" y el conjunto STIX permiten que los gráficos tengan una apariencia similar a los documentos científicos.

**`from scipy import integrate`**: Importa el submódulo `integrate` de `scipy`, que incluye funciones para realizar integraciones numéricas y otros cálculos avanzados relacionados. Es útil para resolver problemas de integración en ciencia e ingeniería.


## Oscilador armónico

In [3]:
t, omega0 = sympy.symbols("t, omega_0", positive=True) #Declaracion de variables
gamma = sympy.symbols("gamma", complex=True) #Hago que gamma sea un valor algebraico y 
                                            #activo que pueda ser complejo

In [4]:
x = sympy.Function("x") # Defino a x como funcion

In [5]:
ode = x(t).diff(t, 2) + 2 * gamma * omega0 * x(t).diff(t) + omega0**2 * x(t) #defino la ED

In [6]:
ode #Muestro la ED

In [7]:
ode_sol = sympy.dsolve(ode) #Resuelvo la ED

In [9]:
ode_sol #Muestro la solución

In [10]:
ics = {x(0): 1, x(t).diff(t).subs(t, 0): 0}

### Documentación

`ics` es un diccionario que se utiliza para definir **condiciones iniciales** en ecuaciones diferenciales:

```python
ics = {x(0): 1, x(t).diff(t).subs(t, 0): 0}
```

Argumentos de la función:
- `x(0): 1` indica que el valor de la función \( x(t) \) en \( t = 0 \) es igual a 1, es decir, \( x(0) = 1 \).
- `x(t).diff(t).subs(t, 0): 0` indica que la derivada de \( x(t) \) con respecto a \( t \), evaluada en \( t = 0 \), es igual a 0, es decir, \( x'(0) = 0 \).

In [11]:
ics

In [ ]:
def apply_ics(sol, ics, x, known_params):
    """
    Apply the initial conditions (ics), given as a dictionary on
    the form ics = {y(0): y0: y(x).diff(x).subs(x, 0): yp0, ...}
    to the solution of the ODE with indepdendent variable x.
    The undetermined integration constants C1, C2, ... are extracted
    from the free symbols of the ODE solution, excluding symbols in
    the known_params list.
    """
    free_params = sol.free_symbols - set(known_params)
    eqs = [(sol.lhs.diff(x, n) - sol.rhs.diff(x, n)).subs(x, 0).subs(ics)
           for n in range(len(ics))]
    sol_params = sympy.solve(eqs, free_params)
    return sol.subs(sol_params)

In [ ]:
x_t_sol = apply_ics(ode_sol, ics, t, [omega0, gamma])

In [ ]:
x_t_sol

In [ ]:
x_t_critical = sympy.limit(x_t_sol.rhs, gamma, 1)

In [ ]:
x_t_critical

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))

tt = np.linspace(0, 3, 250)
for g in [0.1, 0.5, 1, 2.0, 5.0]:
    if g == 1:
        x_t = sympy.lambdify(t, x_t_critical.subs({omega0: 2.0 * sympy.pi}), 'numpy')
    else:
        x_t = sympy.lambdify(t, x_t_sol.rhs.subs({omega0: 2.0 * sympy.pi, gamma: g}), 'numpy')
    ax.plot(tt, x_t(tt).real, label=r"$\gamma = %.1f$" % g)

ax.set_xlabel(r"$t$", fontsize=18)
ax.set_ylabel(r"$x(t)$", fontsize=18)
ax.set_xlim(0, 3)
ax.legend()

fig.tight_layout()
#fig.savefig('harmonic-oscillator.pdf')